In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
# load and display an image with Matplotlib
from matplotlib import image
from matplotlib import pyplot
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from numpy import asarray
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input


# Task 1 - Getting the data in

# The pictures was divided by taken the entire set of pictures taking 80% for train, 20 for test, and 10% from the train set to validation.
# Dette betyder at vi fra det originale data har:
# test: 520 pictures, 260 of each
# train: 1664 pictures, 832 of each
# Validation: 416 pictures, 206 of each

# Task 2 - Create the datagenerators

Due to the high amount of data, you need to write a datagenerator to load the images. This requires you to use the _ImageDataGenerator_ (as shown in Keras Intro - 2) in which you must apply at least one on-the-fly data augmentation. Which type of data augmentation is up to you, but you need to justify your choice in your report. On top of the _ImagaDataGenerator_ you need to apply the _.flow_from_directory_ function to make it work with the directory tree in Task 1

We need to use (at least) one on-the-fly augmentation. Which is better to use? Zooming, flipping, shifting, changing brightness? Should we use more than one? 

ImageDataGenerator: 
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
flow_from_directory: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory

In [47]:
os.getcwd()

'C:\\Users\\jakob\\OneDrive\\Kandidat SDU\\DATASCIENCE\\DS809 Deep learning\\Deep_learning_Data'

In [48]:
train_datagen = ImageDataGenerator(rescale=1./255.0, shear_range=0.2, zoom_range=0.2, horizontal_flip=True) 
validation_datagen = ImageDataGenerator(rescale=1./255.0) 
test_datagen = ImageDataGenerator(rescale=1./255.0)

train_generator = train_datagen.flow_from_directory('Train/', target_size=(150, 150), batch_size=32, class_mode='binary') 
validation_generator = validation_datagen.flow_from_directory('Validation/', target_size=(150, 150), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory('Test/', target_size=(150, 150), class_mode='binary')

Found 1664 images belonging to 2 classes.
Found 416 images belonging to 2 classes.
Found 520 images belonging to 2 classes.


In [57]:
cnn_model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(3,3), 
                input_shape=(150,150,3), 
                activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.SpatialDropout2D(0.2),
        tf.keras.layers.Conv2D(64, kernel_size=(3,3), 
                activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.SpatialDropout2D(0.2),
        tf.keras.layers.Conv2D(128, kernel_size=(3,3), 
                activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.SpatialDropout2D(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, 
                activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1,
                activation='sigmoid'),
        ], name='classifier')

In [58]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9) #additional 0
cnn_model.compile(
    optimizer=opt,
    loss='binary_crossentropy',
    metrics=['accuracy']
)
cnn_model.summary()

Model: "classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
spatial_dropout2d_18 (Spatia (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
spatial_dropout2d_19 (Spatia (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 34, 34, 128)       7

In [59]:
history = cnn_model.fit(train_generator, steps_per_epoch=1664//32, epochs=300, batch_size=32, validation_data=validation_generator, validation_steps=416//32)

Epoch 1/25
52/52 [==============================] - 8s 152ms/step - loss: 0.6981 - accuracy: 0.5210 - val_loss: 0.6907 - val_accuracy: 0.5192
Epoch 2/25
52/52 [==============================] - 8s 149ms/step - loss: 0.6947 - accuracy: 0.5072 - val_loss: 0.6888 - val_accuracy: 0.5553
Epoch 3/25
52/52 [==============================] - 8s 151ms/step - loss: 0.6907 - accuracy: 0.5325 - val_loss: 0.6875 - val_accuracy: 0.5288
Epoch 4/25
52/52 [==============================] - 8s 150ms/step - loss: 0.6911 - accuracy: 0.5288 - val_loss: 0.6861 - val_accuracy: 0.5505
Epoch 5/25
52/52 [==============================] - 8s 156ms/step - loss: 0.6915 - accuracy: 0.5325 - val_loss: 0.6842 - val_accuracy: 0.5649
Epoch 6/25
52/52 [==============================] - 8s 150ms/step - loss: 0.6849 - accuracy: 0.5445 - val_loss: 0.6945 - val_accuracy: 0.4928
Epoch 7/25
52/52 [==============================] - 8s 151ms/step - loss: 0.6897 - accuracy: 0.5343 - val_loss: 0.6830 - val_accuracy: 0.5625
Epoch 

KeyboardInterrupt: 

In [60]:
cnn_model.evaluate(test_generator, steps=len(test_generator))

17/17 [==============================] - 1s 31ms/step - loss: 0.6581 - accuracy: 0.6192


[0.6581153869628906, 0.6192307472229004]

In [61]:
i = 0
for elem in cnn_model.predict(test_generator):
    if elem > 0.5:
        i+=1
print(i)
print(len(test_generator))

314
17
